# Poland 2D Line 001 apply acquisition geometry

Using the "dropped auxilliary channels" from a previous example and the positioning files, this example illustrates how to extract values from the dataframe and apply to headers in segy.

In [3]:
import os
import io
import math
import pathlib
from poland2d_context import segytools

from segytools.segy_trace_header import COORDINATE_SCALAR_MULTIPLIER

from positioning.sps_parser import SPS
from positioning.rps_parser import RPS
from positioning.xps_parser import XPS

def midpoint_coordinates(sx,sy,gx,gy):
    mx = (float(sx) + float(gx)) / 2.0
    my = (float(sy) + float(gy)) / 2.0
    return mx, my

def euclidean_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx**2 + dy**2)

def azimuth_degrees(sx, sy, gx, gy):
    # TODO: fix so that the azimuth is always source to receiver; atan2 {-PI, PI}
    return (180.0 / math.pi) * math.atan2(sx - gx, sy - gy)

sps = SPS(f="data/Line_001.SPS")
sps_df = sps.dataframe()

rps = RPS(f="data/Line_001.RPS")
rps_df = rps.dataframe()

xps = XPS(f="data/Line_001.XPS")
xps_df = xps.dataframe()

In [4]:
TEXTENCODE = 'ebcdic'
BYTEORDER = '>'

def apply_geometry(input_segy_file:str, output_segy_context_manager:io.BufferedReader):
    path_segy_file = pathlib.Path(str(input_segy_file))
    assert (path_segy_file.is_file())
    print(f"reading {path_segy_file.name}")

    segy_file_size = os.path.getsize(input_segy_file)
    print(f"segy file size {segy_file_size}")
    
    file_header_input = segytools.SegyFileHeaderRev2(segy_logger=None)
    trace_header_input = segytools.SegyTraceHeaderRev2(segy_logger=None)
    
    # 'rb' is "read bytes"
    with open(path_segy_file, 'rb') as fobj:
        # read the first 3200 bytes.
        # This will always be 3200 byte textual file header
        b_text_header = fobj.read(3200)
        
        b_file_header = fobj.read(file_header_input.byte_length)
        file_header_input.set_header_values(buf=b_file_header, byteorder=BYTEORDER)
        
        sample_size_in_bytes = file_header_input.sample_format_size_in_bytes()
        trc_data_length_in_bytes = file_header_input.num_samples_per_trace.value * sample_size_in_bytes
    
        output_segy_context_manager.write(b_text_header)
        output_segy_context_manager.write(b_file_header)
        
        # Loop through traces ...
        while fobj.tell() < segy_file_size:
            # TRACE HEADER
            b_trace_header = fobj.read(trace_header_input.byte_length)
            trace_header_input.set_header_values(buf=b_trace_header, byteorder=BYTEORDER)
            if trace_header_input.num_samples.value != file_header_input.num_samples_per_trace.value:
                trc_data_length_in_bytes = file_header_input.num_samples_per_trace.value * sample_size_in_bytes
            # fix incorrect coordinate scalars
            trace_header_input.xy_scalar.value = int(-100)
            trace_header_input.z_scalar.value = int(1)
            # apply acquisition geometry to headers
            trace_header_input.shotpoint.value = xps.source_point(ffid=trace_header_input.field_record_number.value)
            # scale x,y coords and cast to int before assigning to header value; NOTE That the xy scalers in the headers are not correct
            src_x = round(sps.source_x(source_point=trace_header_input.shotpoint.value) / trace_header_input.xy_scalar.mapped_value)
            #--print(sps.source_x(source_point=trace_header_input.shotpoint.value))
            #--print(trace_header_input.xy_scalar.value, trace_header_input.xy_scalar.mapped_value)
            #--print(src_x)
            trace_header_input.src_x_coord.value = int(src_x)
            src_y = round(sps.source_y(source_point=trace_header_input.shotpoint.value) / trace_header_input.xy_scalar.mapped_value)
            trace_header_input.src_y_coord.value = int(src_y)
            src_z = round(sps.source_elevation(source_point=trace_header_input.shotpoint.value))
            trace_header_input.src_elevation.value = int(src_z)
            src_stat = sps.source_static(source_point=trace_header_input.shotpoint.value)  # assuming milliseconds in SPS file
            trace_header_input.src_static_correction.value = int(src_stat)
            #--print(trace_header_input.trc_num_within_field_record.value)
            trace_header_input.group_num.value = xps.reciever_station(ffid=trace_header_input.field_record_number.value, chan=trace_header_input.trc_num_within_field_record.value)
            rcv_x = round(rps.receiver_x(receiver_point=trace_header_input.group_num.value) / trace_header_input.xy_scalar.mapped_value)
            trace_header_input.rcv_x_coord.value = int(rcv_x)
            rcv_y = round(rps.receiver_y(receiver_point=trace_header_input.group_num.value) / trace_header_input.xy_scalar.mapped_value)
            trace_header_input.rcv_y_coord.value = int(rcv_y)
            rcv_z = round(rps.receiver_elevation(receiver_point=trace_header_input.group_num.value))
            trace_header_input.rcv_elevation.value = int(rcv_z)
            rcv_stat = round(rps.receiver_static(receiver_point=trace_header_input.group_num.value))
            trace_header_input.rcv_static_correction.value = int(rcv_stat)
            # calculate the midpoint coordinates and offset
            cmpx, cmpy = midpoint_coordinates(sx=trace_header_input.src_x_coord.value * 0.01, 
                                              sy=trace_header_input.src_y_coord.value * 0.01, 
                                              gx=trace_header_input.rcv_x_coord.value * 0.01, 
                                              gy=trace_header_input.rcv_y_coord.value * 0.01)
            trace_header_input.ens_x_coord.value = int(round(cmpx * 100))
            trace_header_input.ens_y_coord.value = int(round(cmpy * 100))
            offset = euclidean_distance(x1=trace_header_input.src_x_coord.value * 0.01, 
                                        y1=trace_header_input.src_y_coord.value * 0.01, 
                                        x2=trace_header_input.rcv_x_coord.value * 0.01, 
                                        y2=trace_header_input.rcv_y_coord.value * 0.01)
            trace_header_input.offset.value = int(round(offset))
            
            # TRACE DATA
            b_trace_data = fobj.read(trc_data_length_in_bytes)
            # write header, data
            output_segy_context_manager.write(trace_header_input.to_bytes(byteorder=BYTEORDER))
            output_segy_context_manager.write(b_trace_data)

        fobj.close()


path_segy_file_in = pathlib.Path.cwd().joinpath("data", "Line_001_NO_AUX.sgy")
path_segy_file_out = pathlib.Path.cwd().joinpath("data", "Line_001_NO_AUX_GEOM.sgy")
with open(path_segy_file_out, 'ab') as fout:
    apply_geometry(input_segy_file=str(path_segy_file_in), output_segy_context_manager=fout)
    fout.close()

reading Line_001_NO_AUX.sgy
segy file size 441966408
